In [1]:
import torch
import pickle

with open( "x_train_hist.p", "rb" ) as file:
    x_train_hist = pickle.load(file)
    
with open( "y_train_hist.p", "rb" ) as file:
    y_train_hist = pickle.load(file)
    
with open( "x_valid_hist.p", "rb" ) as file:
    x_valid_hist = pickle.load(file)
    
with open( "y_valid_hist.p", "rb" ) as file:
    y_valid_hist = pickle.load(file)

In [2]:
import torch
import random
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

from mriqa_dataset import MRIQADataset
from networks import ClassicCNN, PhilsClassicCnn, CatNet

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Link: https://arxiv.org/abs/2003.04696



In [9]:
# set random seeds for reproducibility
random.seed(21062020)
np.random.seed(21062020)
torch.manual_seed(21062020)
torch.cuda.manual_seed(21062020)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(torch.cuda.current_device())
torch.cuda.set_device(0)

net = ClassicCNN(num_classes=5)
net = net.cuda()

optimizer = optim.Adam(net.parameters())
ce = CrossEntropyLoss().cuda()
num_epochs = len(x_train_hist)
num_mini_batches = 13

loss_csv = open('losses.csv', 'w')
loss_csv.write('epoch,training,validation\n')

best_val_loss = 999
print("start training")
for epoch in range(num_epochs):
    epoch_loss = 0.0
    net.train()
    
    train_batches = x_train_hist[epoch]
    train_labels = y_train_hist[epoch]

    # train loop
    for sample, label in tqdm(zip(train_batches, train_labels), total=len(train_batches), leave=False):       
        sample = sample.cuda()
        label = label.cuda()

        prediction = net(sample)
        loss = ce(prediction, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print('[{}] train-loss: {}'.format(epoch, epoch_loss / num_mini_batches))
    loss_csv.write(str(epoch) + ',' + str(epoch_loss / num_mini_batches))
    loss_csv.flush()

    # validation loop
    net.eval()
    mean_validation_loss = 0
    num_validation_mini_batches = 11

    if epoch % 3 == 0:
        valid_batches = x_valid_hist[epoch//5]
        valid_labels = y_valid_hist[epoch//5]
        
        with torch.no_grad():
            for sample, label in tqdm(zip(valid_batches, valid_labels), total=len(valid_batches), leave=False):              
                sample = sample.cuda()
                label = label.cuda()

                prediction = net(sample)
                validation_loss = ce(prediction, label)

                mean_validation_loss += validation_loss.item()
                
        print('[{}] validation-loss: {}'.format(epoch, mean_validation_loss / num_validation_mini_batches))
        loss_csv.write(',' + str(mean_validation_loss / num_validation_mini_batches) + '\n')
        loss_csv.flush()

    # save best model
    if mean_validation_loss <= best_val_loss:
        torch.save({'epoch': epoch,
                    'model_state_dict': net.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': validation_loss.item()}, 'checkpoint_best_default')
        best_val_loss = mean_validation_loss

print('DONE.')

0
start training


[0] train-loss: 0.42341728393848127


[0] validation-loss: 0.4380675662647594


[1] train-loss: 0.3507945720966046


[2] train-loss: 0.38625192642211914


...

[306] validation-loss: 0.1426882418719205


[307] train-loss: 0.19191545706528884


[308] train-loss: 0.37299243761942935


[309] train-loss: 0.20720622860468352


[309] validation-loss: 0.14478055997328323
DONE.


In [10]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [11]:
prediction

tensor([[ -8.9970,  -4.4122, -16.9409,  15.0178,  -2.2451],
        [ -0.8023, -13.6114,  -1.6850,  17.6273, -11.4828],
        [ -0.9889,   0.5246,  -2.2444,  -2.6369,   3.2851],
        [  2.7983,  -5.5457,   3.3898,  -2.1274,  -1.0825],
        [  1.2322,  -1.0881,   1.8989,  -3.3888,   1.4466],
        [  5.0590,  -4.1397,   1.3992,  -4.7918,  -2.9194],
        [ -3.7646,  -3.1720,  -5.9572,   6.3695,   0.4226],
        [ -2.4624,   5.6199, -16.9658,  -2.3868,   0.4178],
        [  1.1986,  -0.1129,  -2.8071,  -3.4948,  -0.1528],
        [  1.0674,  -0.9836,   2.2941,  -4.1562,   2.2164],
        [  1.6949,  -2.7851,   3.0106,  -3.4684,   1.0700],
        [  1.4913,  -2.0166,   3.0791,  -5.3501,   2.5055],
        [ -1.8201,   0.4987,  -1.5850,  -3.0029,   4.8967],
        [  2.6256,  -5.0120,   3.5869,  -2.6930,  -0.3711],
        [  4.7152,  -2.3928,   0.4477,  -5.8855,  -2.4076],
        [  3.7542,   0.1290,  -2.8193,  -5.3375,  -2.7252],
        [  1.1781,  -0.8517,   1.7547,  

In [12]:
label

tensor([3, 3, 1, 2, 2, 0, 3, 1, 1, 4, 2, 2, 4, 2, 0, 1, 2, 3, 3, 1, 3],
       device='cuda:0')

In [13]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_train_hist, y_train_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [14]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.69      0.93      0.79      6339
           1       0.99      0.38      0.55      6175
           2       0.86      0.92      0.89      6165
           3       1.00      0.99      1.00      6208
           4       0.73      0.86      0.79      6113

    accuracy                           0.82     31000
   macro avg       0.85      0.82      0.80     31000
weighted avg       0.85      0.82      0.80     31000



In [15]:
confusion_matrix(all_labels, all_predictions)

array([[5888,    4,  322,    0,  125],
       [1961, 2331,  104,    1, 1778],
       [ 425,    0, 5696,    0,   44],
       [   6,    0,    0, 6163,   39],
       [ 284,   15,  539,    0, 5275]], dtype=int64)

In [16]:
confusion_matrix(all_labels, all_predictions)

array([[5888,    4,  322,    0,  125],
       [1961, 2331,  104,    1, 1778],
       [ 425,    0, 5696,    0,   44],
       [   6,    0,    0, 6163,   39],
       [ 284,   15,  539,    0, 5275]], dtype=int64)

In [17]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [18]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.69      0.94      0.80      2727
           1       0.99      0.38      0.55      2714
           2       0.87      0.92      0.90      2710
           3       1.00      0.99      0.99      2714
           4       0.72      0.87      0.79      2735

    accuracy                           0.82     13600
   macro avg       0.85      0.82      0.80     13600
weighted avg       0.85      0.82      0.80     13600



In [19]:
print(confusion_matrix(all_labels, all_predictions))

[[2551    3  116    0   57]
 [ 811 1028   48    0  827]
 [ 186    0 2502    0   22]
 [   2    0    0 2682   30]
 [ 132    5  208    0 2390]]


In [20]:
torch.save({'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': validation_loss.item()}, 'final_default')

In [22]:
model = torch.load("checkpoint_best_default")

In [26]:
model = ClassicCNN(num_classes=5)
optim.Adam(net.parameters())

checkpoint = torch.load("checkpoint_best_default")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
model.cuda()

ClassicCNN(
  (block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

In [27]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = model(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [28]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.66      0.94      0.78      2727
           1       1.00      0.33      0.50      2714
           2       0.85      0.91      0.88      2710
           3       1.00      0.99      1.00      2714
           4       0.73      0.86      0.79      2735

    accuracy                           0.81     13600
   macro avg       0.85      0.81      0.79     13600
weighted avg       0.85      0.81      0.79     13600



In [29]:
print(confusion_matrix(all_labels, all_predictions))

[[2563    1  117    0   46]
 [ 959  897   57    0  801]
 [ 212    0 2479    0   19]
 [   2    0    0 2691   21]
 [ 140    3  250    0 2342]]
